In [1]:
import os
import imageio

import pandas as pd
import numpy as np

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.plotly import image as p_image

%matplotlib inline
init_notebook_mode(connected=True)

# Settings

In [2]:
plot_type = 'all' # 'gender' for male-female plots

# Data

In [3]:
# read all recordings
reactions_df = pd.DataFrame()
for filename in os.listdir('./result_emotions/'):
    reaction_df = pd.read_csv('./result_emotions/' + filename)
    reaction_df['id'] = filename
    reaction_df['time_second'] = [x.split('.')[0] for x in reaction_df['time'].astype('str').values]
    reaction_df['time_second'] = reaction_df['time_second'].astype('int')
    reactions_df = reactions_df.append(reaction_df)
    
# group recordings by seconds 
reaction_df_sec = pd.DataFrame()
for id_f, frame_df in reactions_df.groupby(['id']):
    f_df = frame_df.copy()
    
    f_df_sec = f_df.groupby(['id', 'time_second'], as_index = False)['reaction'].agg(lambda x:x.value_counts().index[0])
    f_df_sec = f_df_sec.merge(f_df[['id',  'gender', 'time_second', 'reaction', 'reaction_tense']], on = ['id', 'time_second', 'reaction']).groupby(['id',  'gender', 'time_second', 'reaction'], as_index = False)['reaction_tense'].mean()
    
    f_df_sec = f_df_sec.tail(69)
    f_df_sec.time_second = range(f_df_sec.shape[0])
    
    reaction_df_sec = reaction_df_sec.append(f_df_sec)
    
reaction_df_sec.head()

,id,gender,time_second,reaction,reaction_tense
11,female.csv,female,0,neutral,0.662856
12,female.csv,female,1,neutral,0.718655
13,female.csv,female,2,neutral,0.617910
14,female.csv,female,3,neutral,0.540128
15,female.csv,female,4,neutral,0.576132


# Visualize

In [4]:
# backup
reaction_df = reaction_df_sec.copy() 

# settings
max_second = reaction_df.time_second.max()
first_second = reaction_df_sec.shape[0]

# singe frame
new_reaction_df = pd.DataFrame()
for id_f, frame_df in reaction_df.groupby(['id']):
    first_second = frame_df.shape[0]
    frame_df = frame_df[:first_second].copy()
    frame_df['graph_tense'] = frame_df['reaction_tense']
    frame_df.loc[frame_df[frame_df.reaction == 'surprise'].index, 'reaction'] = 'neutral'
    frame_df.loc[frame_df[frame_df.reaction == 'sad'].index, 'reaction'] = 'angry'
    frame_df.loc[frame_df[frame_df.reaction == 'happy'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'happy']['graph_tense'] * 33 + 66
    frame_df.loc[frame_df[frame_df.reaction == 'neutral'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'neutral']['graph_tense'] * 33 + 33
    frame_df.loc[frame_df[frame_df.reaction == 'angry'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'angry']['graph_tense'] * 33

    new_reaction_df = new_reaction_df.append(frame_df)
new_reaction_df['id'] = new_reaction_df['id'].astype('str')
new_reaction_df.reset_index(inplace=True, drop = True)

# plot df
if plot_type == 'all':

    # all-in-one
    new_reaction_df= new_reaction_df.groupby('time_second', as_index=False)['graph_tense'].mean()
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] < 33)].index, 'reaction'] = 'angry'
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 33) & (new_reaction_df['graph_tense'] < 66)].index, 'reaction'] = 'neutral'
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 66)].index, 'reaction'] = 'happy'
    new_reaction_df['color'] = 'rgb(17,56,117)'
    new_reaction_df.head()
    
elif plot_type == 'gender':

    # male vs female
    new_reaction_df= new_reaction_df.groupby(['time_second', 'gender'], as_index=False)['graph_tense'].mean()
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] < 33)].index, 'reaction'] = 'angry'
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 33) & (new_reaction_df['graph_tense'] < 66)].index, 'reaction'] = 'neutral'
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 66)].index, 'reaction'] = 'happy'
    new_reaction_df.loc[new_reaction_df[new_reaction_df.gender == 'male'].index, 'color'] = 'rgb(17,56,117)'
    new_reaction_df.loc[new_reaction_df[new_reaction_df.gender == 'female'].index, 'color'] = 'rgb(166,28,0)'
    new_reaction_df.head()

In [8]:
x = sorted(new_reaction_df.time_second.unique())
y4 = [0 for x in range(frame_df.shape[0])]
y1 = [33 for x in range(frame_df.shape[0])]
y2 = [66 for x in range(frame_df.shape[0])]
y3 = [100 for x in range(frame_df.shape[0])]

trace1 = go.Scatter(
    x=x,
    y=y1,
    fill='tozeroy',
    line=dict(
        width=0.5,
        color='rgb(230,184,175)',
    )
)
trace2 = go.Scatter(
    x=x,
    y=y2,
    fill='tonexty',
    opacity=0.5,
    line = dict(
        width=0.5,
        color='rgb(255,242,204)'
    )
)
trace3 = go.Scatter(
    x=x,
    y=y3,
    fill='tonexty',
    opacity=0.5,
    line=dict(
        width=0.5,
        color='rgb(182,215,168)',
    )
)
trace4 = go.Scatter(
    x=x,
    y=y4,
    fill='tonexty',
    opacity=0.5,
    line=dict(
        width=0.5,
        color='rgb(230,184,175)',
    )

)
data = [trace4, trace1, trace2, trace3]

# plot df
if plot_type == 'all':

    trace = go.Scatter(
            x = new_reaction_df.time_second.values,
            y = new_reaction_df.graph_tense.values,
            line=dict(
                width=3,
                color=new_reaction_df.color.values[0],
            )
        )

    data.append(trace)
    
elif plot_type == 'gender':
    for id_f, frame_df in new_reaction_df.groupby(['gender']):

        trace = go.Scatter(
            x = frame_df.time_second.values,
            y = frame_df.graph_tense.values,
            line=dict(
                width=3,
                color=frame_df.color.values[0],
            )
        )

        data.append(trace)
    
layout = go.Layout(
    title='Анализ ролика',
    showlegend=False,
    xaxis=dict(
        title='Секунды',
        range=[min(x) - 1, max(x) + 1]
    ),
    yaxis=dict(
        title='Настроениие зрителя',
        type='linear',
        range=[0, 100],
        ticksuffix='%'
    ),
) 


fig = go.Figure(data=data, layout=layout)
a = plot(fig, filename='graphs/plot_%s.html' % plot_type, show_link=False, auto_open=False)